In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.shape

(360336, 17)

In [ ]:

nan_counts = train.isna().sum()


nan_columns = nan_counts[nan_counts > 0]
if not nan_columns.empty:
    print("Columns with NaN values:\n", nan_columns)
else:
    print("No NaN values found in train_df.")


No NaN values found in train_df.


In [ ]:
train.head(10)

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.09
1,1,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,0.0,121.80
2,2,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,0.0,83.51
3,3,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,0.0,66.78
4,4,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,1.0,111.51
5,5,9.08,4.0,5.0,5.0,3.0,7.26,0.0,1.0,5.0,33858.0,1.0,0.0,1.0,1.0,1.0,142.58
6,6,4.80,2.0,1.0,0.0,2.0,9.58,0.0,0.0,6.0,27694.0,1.0,1.0,1.0,1.0,1.0,69.47
7,7,4.29,3.0,2.0,0.0,2.0,16.90,1.0,0.0,2.0,23688.0,1.0,1.0,1.0,1.0,1.0,68.84
8,8,8.55,3.0,5.0,0.0,2.0,13.80,1.0,0.0,6.0,38382.0,0.0,0.0,0.0,0.0,0.0,87.07
9,9,3.08,4.0,1.0,0.0,3.0,15.70,1.0,1.0,9.0,27694.0,1.0,1.0,1.0,1.0,1.0,80.29


In [ ]:
test = pd.read_csv('test.csv')
test.head()

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist
0,360336,7.24,4.0,1.0,0.0,2.0,10.80,0.0,1.0,7.0,20319.0,0.0,0.0,0.0,0.0,0.0
1,360337,6.90,2.0,2.0,2.0,3.0,8.51,1.0,0.0,4.0,33858.0,1.0,0.0,1.0,1.0,1.0
2,360338,8.34,3.0,0.0,0.0,3.0,8.77,0.0,1.0,14.0,39696.0,0.0,0.0,1.0,1.0,0.0
3,360339,5.48,2.0,3.0,3.0,2.0,21.90,1.0,0.0,9.0,23688.0,1.0,1.0,1.0,1.0,1.0
4,360340,4.80,3.0,2.0,0.0,2.0,10.90,1.0,0.0,11.0,27694.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Extract test 'id' column for submission purposes
test_ids = test['id'].values

# Step 2: Feature Engineering
print("Feature engineering...")
train['log_cost'] = np.log1p(train['cost'])
target = 'log_cost'

# Create new features
for df in [train, test]:
    df['children_x_avg_cars'] = df['total_children'] * df['avg_cars_at home(approx).1']

    # Calculate sum of important features (excluding children_x_avg_cars)
    important_features_sum = (
        df['coffee_bar'] +
        df['video_store'] +
        df['salad_bar'] +
        df['florist'] +
        df['prepared_food'] + 1
    )

    # Create the new feature by dividing store_sqft by the sum
    df['sqft_div_sum_features'] = df['store_sqft'] / important_features_sum

# Define the most important features for the model
most_important_features = [
    'total_children', 'num_children_at_home', 'avg_cars_at home(approx).1',
    'store_sqft', 'coffee_bar', 'video_store', 'salad_bar', 'florist',
    'prepared_food', 'children_x_avg_cars', 'sqft_div_sum_features'
]

# Step 3: Preprocess Data
print("Preprocessing data...")

preprocessor = Pipeline([
    ('ohe', ColumnTransformer([
        ('onehot', OneHotEncoder(drop='first', sparse_output=False),
         ['total_children', 'num_children_at_home', 'avg_cars_at home(approx).1', 'store_sqft', 'prepared_food'])
    ], remainder='passthrough')),
    ('scaler', StandardScaler())
])

# Transform the data
X = preprocessor.fit_transform(train[most_important_features]).astype(float)
y = train[target]
X_te = preprocessor.transform(test[most_important_features]).astype(float)

print(f"Shape of transformed training features: {X.shape}")
print(f"Shape of transformed test features: {X_te.shape}")


Feature engineering...
Preprocessing data...
Shape of transformed training features: (360336, 40)
Shape of transformed test features: (240224, 40)


In [ ]:
!pip install catboost


# Approach 1

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Define the LightGBM model with DART-specific parameters
dart_params = {
    'boosting_type': 'dart',
    'learning_rate': 0.3,
    'n_estimators': 400,
    'num_leaves': 200,
    'min_child_samples': 1,
    'min_child_weight': 10
}

lgbm_model = lgb.LGBMRegressor(**dart_params)

# Cross-validation setup with 10 folds
kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=42)
mse_scores = []  # To store MSE of each fold

# Training and evaluating on each fold
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Fit the LightGBM model on the current fold
    lgbm_model.fit(X_train, y_train)

    # Predictions and evaluation for the current fold
    val_preds = lgbm_model.predict(X_val)
    mse = mean_squared_error(y_val, val_preds)
    mse_scores.append(mse)
    print(f"MSE for fold: {mse}")

# Average MSE across all folds
mean_mse = np.mean(mse_scores)
print(f"Average Mean Squared Error across 10 folds: {mean_mse}")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
import lightgbm as lgb

# Assuming you have already defined your models and preprocessor
# and fitted your stacked model

# Make predictions on the test set

final_test_preds = lgbm_model.predict(X_te)

# Reverse the log1p transformation to get actual values
final_test_preds = np.expm1(final_test_preds)

# Create a DataFrame with predictions
submission = pd.DataFrame({
    'id': test_ids,
    'cost': final_test_preds
})

# Display a sample of predictions
print("Sample of predictions:")
print(submission.head(15))

# Save the submission file
submission.to_csv('nn_submission.csv', index=False)
print("Submission file 'nn_submission.csv' created successfully!")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 130
[LightGBM] [Info] Number of data points in the train set: 360336, number of used features: 39
[LightGBM] [Info] Start training from score 4.563348
Sample of predictions:
        id        cost
0   360336   90.519338
1   360337   93.912774
2   360338   94.010640
3   360339   99.393212
4   360340   78.129393
5   360341  104.343428
6   360342  110.603090
7   360343   90.845932
8   360344   90.708952
9   360345   90.985704
10  360346  100.911235
11  360347   93.063684
12  360348   88.958485
13  360349   93.127619
14  360350   88.760927
Submission file 'nn_submission.csv' created successfully!


# Approach 2

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Define the DART model parameters
dart_params = {
    'boosting_type': 'dart',
    'learning_rate': 0.3,
    'n_estimators': 400,
    'num_leaves': 200,
    'min_child_samples': 1,
    'min_child_weight': 10
}

# Define the GBDT model parameters, tuned for better performance with GBDT
gbdt_params = {
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,       # Lower learning rate for GBDT
    'n_estimators': 700,         # Higher number of trees for GBDT
    'num_leaves': 130,           # Fewer leaves than DART for generalization
    'min_child_samples': 5,      # Slightly higher min child samples
    'min_child_weight': 5,       # Adjusted weight for stability
    'subsample': 0.8,            # Subsample ratio for diversity
    'colsample_bytree': 0.8      # Feature subsampling for GBDT
}

# Instantiate the LightGBM models
dart_model = lgb.LGBMRegressor(**dart_params)
gbdt_model = lgb.LGBMRegressor(**gbdt_params)

# # Cross-validation setup with 10 folds
# kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=42)
# mse_scores = []  # To store MSE of each fold

# # Training and evaluating on each fold
# for train_index, val_index in kf.split(X):
#     X_train, X_val = X[train_index], X[val_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#     # Fit both models on the current fold
#     dart_model.fit(X_train, y_train)
#     gbdt_model.fit(X_train, y_train)

#     # Make predictions from both models
#     dart_preds = dart_model.predict(X_val)
#     gbdt_preds = gbdt_model.predict(X_val)

#     # Average the predictions
#     combined_preds = (dart_preds + gbdt_preds) / 2

#     # Evaluate using Mean Squared Error
#     mse = mean_squared_error(y_val, combined_preds)
#     mse_scores.append(mse)
#     print(f"MSE for fold: {mse}")

# # Calculate the average MSE across all folds
# mean_mse = np.mean(mse_scores)
# print(f"Average Mean Squared Error across 10 folds: {mean_mse}")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
import lightgbm as lgb

dart_model.fit(X, y)
gbdt_model.fit(X, y)

dart_test_preds = dart_model.predict(X_te)
gbdt_test_preds = gbdt_model.predict(X_te)

# Average the predictions of both models
final_test_preds = (dart_test_preds + gbdt_test_preds) / 2


# Reverse the log1p transformation to get actual values
final_test_preds = np.expm1(final_test_preds)

# Create a DataFrame with predictions
submission = pd.DataFrame({
    'id': test_ids,
    'cost': final_test_preds
})

# Display a sample of predictions
print("Sample of predictions:")
print(submission.head(15))

# Save the submission file
submission.to_csv('nn_submission.csv', index=False)
print("Submission file 'nn_submission.csv' created successfully!")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016795 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 192
[LightGBM] [Info] Number of data points in the train set: 360336, number of used features: 40
[LightGBM] [Info] Start training from score 4.563348
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 192
[LightGBM] [Info] Number of data points in the train set: 360336, number of used features: 40
[LightGBM] [Info] Start training from score 4.563348
Sample of predictions:
        id        cost
0   360336   90.471951
1   360337   94.034007
2   360338   94.123512
3   360339   99.818457
4   360340   78.190939
5   360341  104.350751
6   